In [1]:
import pandas as pd
import json
import requests
import math

In [6]:
#Global paramenters
#Name of drug changes
base_url = "https://rxnav.nlm.nih.gov/REST/rxcui/"
base_url_2 = "https://rxnav.nlm.nih.gov/REST/"
getAllRelatedInfo = "/allrelated.json"
getRelatedByType = "/related.json?tty=IN"
df = pd.DataFrame()

In [7]:
def query_self(name_of_drug):
    rd_df = pd.read_json(base_url+name_of_drug+getRelatedByType)
    rd_df = pd.DataFrame(rd_df.relatedGroup.conceptGroup)
    rd_df = pd.DataFrame(rd_df.conceptProperties[0])
    return rd_df

In [116]:
def query_drug(name_of_drug):
    row = 0
    df = pd.DataFrame()
    rd_df = pd.read_json(base_url+name_of_drug+getAllRelatedInfo)
    rd_df = pd.DataFrame(rd_df.allRelatedGroup.conceptGroup)
    rd_df = rd_df.loc[rd_df["tty"] != "DF"].reset_index(drop=True)
    rd_df = rd_df.loc[rd_df["tty"] != "DFG"].reset_index(drop=True)
    while row < rd_df.shape[0]:
        if(pd.isna(rd_df.conceptProperties[row]) is True):
            row+=1
        else:
            new_df = pd.DataFrame(rd_df.conceptProperties[row])
            if df.empty:
                df = new_df
            else:
                df = pd.concat([df, new_df]).reset_index(drop=True)
            row += 1
    return df

In [57]:
query_drug('3304').to_csv("test.csv")

In [58]:
fentanyl = "4337"
morphine = "7052"
buprenorphine = "1819"
pethidine = "6754"
dihydromorphinone = "3423"
methadone = "6813"
hydrocodone = "5489"
tapentadol = "787390"
oxycodone = "7804"
oxymorphone = "7814"
codeine = "2670"
dihydrocodeine = "23088"
#Added
tramadol = "10689"
levorphanol = "6378"
nalbuphine = "7238"
pentazocine = "8001"
tilidine = "10597"
butorphanol = "1841"
dezocine = "22713"
levomethadone = "236913"
normethadone = "32009"
dextromoramide = "3290"
ethylmorphine = "4166"
alfentanil = "480"
sufentanil = "56795"
meptazinol = "6761"
remifentanil = "73032"
papaveretum = "7894"
phenazocine = "8119"
pirinitramide = "8354"
propoxyphene = "8785"
opium = "7676"
heroin = "3304"	

In [144]:
diphenoxylate = "3500"
oxycodone_hydrochloride = "82063"

In [145]:
query_drug(diphenoxylate).to_csv("test.csv")

In [123]:
#loop array to get complete list
df_opioid = pd.DataFrame()
drugs = [fentanyl, morphine, buprenorphine, pethidine, dihydromorphinone, methadone, hydrocodone, tapentadol, oxycodone, oxymorphone, codeine, dihydrocodeine, tramadol, levorphanol, nalbuphine, pentazocine, tilidine, butorphanol, dezocine, levomethadone, normethadone, dextromoramide, ethylmorphine, alfentanil, sufentanil, meptazinol, remifentanil, papaveretum, phenazocine, pirinitramide, propoxyphene, opium, heroin]
for drug in drugs:
    self_df = query_self(drug)
    merg_df = query_drug(drug)
    df_opioid = pd.concat([df_opioid, self_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
    df_opioid = pd.concat([df_opioid, merg_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
df_opioid

,rxcui,name,synonym,tty,language,suppress,umlscui
0,4337,fentanyl,,IN,ENG,N,
1,1053648,Abstral,,BN,ENG,N,
2,1115547,Lazanda,,BN,ENG,N,
3,1237051,Subsys,,BN,ENG,N,
4,151678,Duragesic,,BN,ENG,N,
...,...,...,...,...,...,...,...
2586,333004,heroin 10 MG,,SCDC,ENG,N,
2587,371779,heroin Oral Tablet,,SCDF,ENG,N,
2588,1160338,heroin Injectable Product,,SCDG,ENG,N,
2589,1160889,heroin Oral Product,,SCDG,ENG,N,


In [124]:
#Export
df_opioid.to_excel("New_Complete_Opioids_List.xlsx", index=False)

In [125]:
#take list and format it to the RXNorm api format
#This list was made by combinding the excel files
df_list = pd.read_excel("Complete_Opioids_List.xlsx")
df_list = df_list[['Code', 'Description','TTY']]
df_list.rename(columns= {'Code' : 'rxcui', 'Description': 'name'}, inplace = True)
df_list

,rxcui,name,TTY
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,SCD
1,1010603,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,SBD
2,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,SCD
3,1010606,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,SBD
4,1014599,acetaminophen 300 MG / oxycodone hydrochloride...,SCD
...,...,...,...
2254,995842,Biotussin,BN
2255,2179635,tramadol hydrochloride 100 MG Oral Tablet,SCD
2256,2182349,chlorpheniramine maleate 0.4 MG/ML / hydrocodo...,SCD
2257,864826,methadone hydrochloride 5 MG/ML Oral Solution,SCD


In [126]:
#This list was created by pulling from the RxNorm API
df_opioid = df_opioid[['rxcui', 'name',"tty"]]

In [127]:
df_opioid

,rxcui,name,tty
0,4337,fentanyl,IN
1,1053648,Abstral,BN
2,1115547,Lazanda,BN
3,1237051,Subsys,BN
4,151678,Duragesic,BN
...,...,...,...
2586,333004,heroin 10 MG,SCDC
2587,371779,heroin Oral Tablet,SCDF
2588,1160338,heroin Injectable Product,SCDG
2589,1160889,heroin Oral Product,SCDG


In [128]:
same_drug_count = 0
new_drug_count = 0
drug_not_found_count = 0
same_drug = pd.DataFrame({"drugs_in_list_rxcui": [], "drugs_in_list_name": []})
new_drug = pd.DataFrame({"new_drugs_rxcui": [], "new_drugs_name": []})
for code in df_opioid['rxcui']:
    drug_name = df_opioid['name'].where(df_opioid['rxcui'] == code).dropna().values[0]
    if df_list.isin([int(code)]).any().any() == True:
        same_drug_count += 1
        same_drug = same_drug.append({'drugs_in_list_rxcui': code, 'drugs_in_list_name' : drug_name}, ignore_index = True)
    else:
        new_drug_count += 1
        new_drug = new_drug.append({'new_drugs_rxcui': code, 'new_drugs_name': drug_name}, ignore_index = True)
print('Same drug count: ', same_drug_count)
print('New drug count: ', new_drug_count)

Same drug count:  2038
New drug count:  553


In [129]:
#Displays same and new drugs
df_final = same_drug.join(new_drug)
df_final.head()

,drugs_in_list_rxcui,drugs_in_list_name,new_drugs_rxcui,new_drugs_name
0,4337,fentanyl,1115548,fentanyl 0.1 MG/ACTUAT [Lazanda]
1,1053648,Abstral,1115578,fentanyl 0.4 MG/ACTUAT [Lazanda]
2,1115547,Lazanda,1237052,fentanyl 0.1 MG/ACTUAT [Subsys]
3,1237051,Subsys,1237058,fentanyl 0.2 MG/ACTUAT [Subsys]
4,151678,Duragesic,1237061,fentanyl 0.4 MG/ACTUAT [Subsys]


In [130]:
absent_drugs = pd.DataFrame({"drugs_not_found_rxcui": [], "drugs_not_found_name": [], "drugs_not_found_tty": []})
count = 0
#for every code from the combind excel list
for code in df_list['rxcui']:
    drug_name = df_list['name'].where(df_list['rxcui'] == code).dropna().values[0]
    tty = df_list['TTY'].where(df_list['rxcui'] == code).dropna().values[0]
    if df_final.isin([str(code)]).any().any() == True:
        count+=1
    else:
        absent_drugs = absent_drugs.append({'drugs_not_found_rxcui': str(code), 'drugs_not_found_name': drug_name, 'drugs_not_found_tty': tty}, ignore_index = True)
print('Count: ', count)
absent_drugs.head()
        
        

Count:  2038


,drugs_not_found_rxcui,drugs_not_found_name,drugs_not_found_tty
0,1050409,oxycodone hydrochloride 20 MG/ML Oral Solution...,SBD
1,1112224,chlorpheniramine maleate 0.8 MG/ML / hydrocodo...,SBD
2,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SCD
3,1114002,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SBD
4,1114003,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SCD


In [131]:
df_final = pd.DataFrame()
df_final = same_drug.join(new_drug)
df_final = df_final.join(absent_drugs)
print(df_final.shape) #check if correct size
df_final.head()

(2038, 7)


,drugs_in_list_rxcui,drugs_in_list_name,new_drugs_rxcui,new_drugs_name,drugs_not_found_rxcui,drugs_not_found_name,drugs_not_found_tty
0,4337,fentanyl,1115548,fentanyl 0.1 MG/ACTUAT [Lazanda],1050409,oxycodone hydrochloride 20 MG/ML Oral Solution...,SBD
1,1053648,Abstral,1115578,fentanyl 0.4 MG/ACTUAT [Lazanda],1112224,chlorpheniramine maleate 0.8 MG/ML / hydrocodo...,SBD
2,1115547,Lazanda,1237052,fentanyl 0.1 MG/ACTUAT [Subsys],1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SCD
3,1237051,Subsys,1237058,fentanyl 0.2 MG/ACTUAT [Subsys],1114002,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SBD
4,151678,Duragesic,1237061,fentanyl 0.4 MG/ACTUAT [Subsys],1114003,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SCD


In [132]:
#export
df_final.to_excel("Opioids_Comparison.xlsx", index=False)

In [133]:
#start here for absolete
#reduce data to be used to find obsolete
absent_drugs

,drugs_not_found_rxcui,drugs_not_found_name,drugs_not_found_tty
0,1050409,oxycodone hydrochloride 20 MG/ML Oral Solution...,SBD
1,1112224,chlorpheniramine maleate 0.8 MG/ML / hydrocodo...,SBD
2,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SCD
3,1114002,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SBD
4,1114003,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SCD
...,...,...,...
216,71225,apomorphine hydrochloride,PIN
217,89781,"belladonna extract, USP",IN
218,993764,Capital and Codeine,BN
219,2182349,chlorpheniramine maleate 0.4 MG/ML / hydrocodo...,SCD


In [134]:
df_status = pd.DataFrame()
index = 0
for code in absent_drugs['drugs_not_found_rxcui']:
    getRxcuiHistoryStatus = 'rxcui/{}/historystatus.json'.format(code)
    df_type = pd.read_json(base_url_2+getRxcuiHistoryStatus)
    df_type = pd.DataFrame(df_type.rxcuiStatusHistory.metaData, index=[0])
    df_type['rxcui'] = code
    df_status = pd.concat([df_status, df_type]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
df_status

,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate,rxcui
0,Obsolete,RXNORM,012011,,NO,012011,122021,,1050409
1,Obsolete,RXNORM,072011,,NO,072011,052021,,1112224
2,Obsolete,RXNORM,082011,,NO,082011,122021,,1113998
3,Obsolete,RXNORM,082011,,NO,082011,122021,,1114002
4,Obsolete,RXNORM,082011,,NO,082011,032022,,1114003
...,...,...,...,...,...,...,...,...,...
216,Active,RXNORM,042005,,YES,052006,,,71225
217,Active,RXNORM,042005,,YES,052006,,,89781
218,Obsolete,RXNORM,072010,,NO,072010,072021,,993764
219,Obsolete,RXNORM,092019,,NO,092019,122021,,2182349


In [135]:
absent_drugs = absent_drugs.rename(columns={"drugs_not_found_rxcui": "rxcui"})
absent_drugs

,rxcui,drugs_not_found_name,drugs_not_found_tty
0,1050409,oxycodone hydrochloride 20 MG/ML Oral Solution...,SBD
1,1112224,chlorpheniramine maleate 0.8 MG/ML / hydrocodo...,SBD
2,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SCD
3,1114002,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SBD
4,1114003,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SCD
...,...,...,...
216,71225,apomorphine hydrochloride,PIN
217,89781,"belladonna extract, USP",IN
218,993764,Capital and Codeine,BN
219,2182349,chlorpheniramine maleate 0.4 MG/ML / hydrocodo...,SCD


In [136]:
df_not_found = absent_drugs.merge(df_status, on='rxcui')

In [137]:
df_not_found

,rxcui,drugs_not_found_name,drugs_not_found_tty,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate
0,1050409,oxycodone hydrochloride 20 MG/ML Oral Solution...,SBD,Obsolete,RXNORM,012011,,NO,012011,122021,
1,1112224,chlorpheniramine maleate 0.8 MG/ML / hydrocodo...,SBD,Obsolete,RXNORM,072011,,NO,072011,052021,
2,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SCD,Obsolete,RXNORM,082011,,NO,082011,122021,
3,1114002,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SBD,Obsolete,RXNORM,082011,,NO,082011,122021,
4,1114003,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SCD,Obsolete,RXNORM,082011,,NO,082011,032022,
...,...,...,...,...,...,...,...,...,...,...,...
216,71225,apomorphine hydrochloride,PIN,Active,RXNORM,042005,,YES,052006,,
217,89781,"belladonna extract, USP",IN,Active,RXNORM,042005,,YES,052006,,
218,993764,Capital and Codeine,BN,Obsolete,RXNORM,072010,,NO,072010,072021,
219,2182349,chlorpheniramine maleate 0.4 MG/ML / hydrocodo...,SCD,Obsolete,RXNORM,092019,,NO,092019,122021,


In [138]:
df_active=df_not_found.loc[df_not_found['status']== 'Active'].reset_index(drop=True)

In [139]:
df_active

,rxcui,drugs_not_found_name,drugs_not_found_tty,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate
0,1154667,atropine / diphenoxylate Oral Liquid Product,SCDG,Active,RXNORM,102011,,YES,102011,,
1,1154668,atropine / diphenoxylate Oral Product,SCDG,Active,RXNORM,102011,,YES,102011,,
2,1154669,atropine / diphenoxylate Pill,SCDG,Active,RXNORM,102011,,YES,102011,,
3,1161354,paregoric Oral Liquid Product,SCDG,Active,RXNORM,102011,,YES,102011,,
4,1161355,paregoric Oral Product,SCDG,Active,RXNORM,102011,,YES,102011,,
5,1165327,loperamide Oral Liquid Product,SCDG,Active,RXNORM,102011,,YES,102011,,
6,1165328,loperamide Oral Product,SCDG,Active,RXNORM,102011,,YES,102011,,
7,1167781,Imodium Oral Liquid Product,SBDG,Active,RXNORM,102011,,YES,102011,,
8,1167782,Imodium Oral Product,SBDG,Active,RXNORM,102011,,YES,102011,,
9,1176518,Diamode Oral Product,SBDG,Active,RXNORM,102011,,YES,102011,,


In [140]:
#export
df_not_found.to_excel("Status_of_Not_found.xlsx", index=False)
df_active.to_excel("Active_Not_Found.xlsx", index=False)

Most Occuring

In [368]:
most_occuring = pd.Series(' '.join(df_active['name']).lower().split()).value_counts()[:100]

In [371]:
most_occuring = pd.DataFrame(most_occuring)

In [372]:
most_occuring

,0
/,2391
mg,2259
mg/ml,2034
oral,1575
hydrochloride,946
...,...
0.02,41
suppository,41
6,40
hyaluronate,39


In [373]:
most_occuring.to_csv("Most_Occuring.csv")